In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import numpy as np
from sklearn.model_selection import train_test_split
import random
from sklearn.metrics import balanced_accuracy_score, brier_score_loss, accuracy_score, roc_auc_score, roc_curve, auc
from scipy.stats import multivariate_normal, bernoulli, beta, norm

In [ ]:
data = pd.read_csv('../Data/CVD_cleaned.csv')

In [ ]:
display(data.head())

## Visualise the entire data, with one singular plot per column in the dataset:

In [ ]:
def plot_data_columns(data: pd.DataFrame) -> None:
    """Plot the columns of a dataframe, where every
    column is plotted as a single histogram or bar plot.
    A column with only 0 or 1 as its values is plotted
    as a bar plot and will get ticks 'Yes' or 'No'.

    Args:
        data (pd.DataFrame): any pandas dataframe.
    """
    plots_per_row = 3
    num_cols = len(data.columns)
    num_rows = np.ceil(num_cols / plots_per_row).astype(int)

    # Figsize is hand picked
    fig = plt.figure(figsize=(15, 8 * num_rows))

    for i, col in enumerate(data.columns):
        ax = fig.add_subplot(num_rows, plots_per_row, i + 1)

        # If everything is numbers, we make a histogram
        if all([isinstance(x, (int, float)) for x in data[col]]):
            ax.hist(data[col], bins=30, edgecolor='k', color='c')
            m = np.mean(data[col])
            s = np.std(data[col])
            ax.axvline(m, color='red', linestyle='--', label=fr'Mean $\mu$')
            ax.axvline(m + s, color='green', linestyle='--', label=fr'Mean $\mu \mp $ std $\sigma$')
            ax.axvline(m - s, color='green', linestyle='--')
            ax.legend()
            
        else:
        # Otherwise it must be a bar plot
            ax.bar(data[col].unique(), data[col].value_counts(), edgecolor='k', color='c')
            # Check if only 0 and 1, then we must change the ticks
            if all([x in [0, 1] for x in data[col]]):
                ax.set_xticks([0, 1])
                ax.set_xticklabels(['No', 'Yes'])
        
        ax.tick_params(axis='x', rotation=90)
        ax.set_title(col)

    plt.tight_layout()
    plt.show()

plot_data_columns(data)

## Now visualise the data for only females:

In [ ]:
plot_data_columns(data.loc[data["Sex"] == "Female"])

## Now visualise the data for only males:

In [ ]:
plot_data_columns(data.loc[data["Sex"] == "Male"])

## The correlation matrix for our dataset:

In [ ]:
# For the correlation we can only take columns with numbers
data_num = data.select_dtypes(include=[int, float])
corr = data_num.corr()

plt.imshow(corr, cmap='viridis')
plt.colorbar()
plt.xticks(np.arange(len(corr.columns)), corr.columns, rotation=90)
plt.yticks(np.arange(len(corr.columns)), corr.columns)
plt.title('Correlation matrix for the dataset')
plt.show()

## Reformatting the data for logistic regression

In [ ]:
# reformatting the data to be suitable for logistic regression
# transforming categorical variables (Yes=1, No=0)
# transforming 'Sex' variable to binary 'Sex_Male' variable
data = data.replace({'Skin_Cancer':'Yes', 'Other_Cancer': 'Yes', 'Heart_Disease': 'Yes', 'Depression': 'Yes', 'Smoking_History': 'Yes', 'Exercise': 'Yes'
                     , 'Sex': 'Male', 'Arthritis': 'Yes'}, 1).replace({'Skin_Cancer':'No', 'Other_Cancer': 'No', 'Heart_Disease': 'No'
                                                   , 'Depression': 'No', 'Smoking_History': 'No', 'Exercise': 'No'
                                                   , 'Sex': 'Female', 'Arthritis': 'No'}, 0).rename(columns={'Sex':'Sex_Male'})

In [ ]:
data.head()

## Checking columns with multiple categories

In [ ]:
print(data['Age_Category'].unique())

In [ ]:
print(data['Diabetes'].unique())

In [ ]:
print(data['Checkup'].unique())

In [ ]:
print(data['General_Health'].unique())

## Normalisation of numerical values

In [ ]:
# normalize numerical values
def normalize_data(data):
    """
    This function takes a column of data and normalizes it.
    """
    min_value = min(data)
    max_value = max(data)
    normalized_data = []

    for value in data:
        new_value = (value - min_value) / (max_value - min_value)
        normalized_data.append(new_value)

    return normalized_data

# normalise the height (done in such a way that we can interpret an increase in height as a 10 cm increase over the average height)
data['Height_(cm)'] = (data['Height_(cm)']- data['Height_(cm)'].mean())/10
data = data.rename(columns={'Height_(cm)':'Height_norm'})

# normalise the weight (done in such a way that we can interpret an increase in weight as a 5 kg increase over the average weight)
data['Weight_(kg)'] = (data['Weight_(kg)']- data['Weight_(kg)'].mean())/5
data = data.rename(columns={'Weight_(kg)':'Weight_norm'})

# normalise the bmi (done in such a way that we can interpret an increase in bmi as a 1 point increase over the average bmi)
data['BMI'] = (data['BMI']- data['BMI'].mean())
data = data.rename(columns={'BMI':'BMI_norm'})

data.head(10)

## Transforming the remaining columns (multiple categories)

In [ ]:
# set age to numerical by taking the mean of the existing categories (ASSUMPTION: 80+ is estimated to 85)
age_mapping = {'18-24': 21.0, '25-29': 27.0, '30-34': 32.0, '35-39': 37.0,'40-44': 42.0, '45-49': 47.0, '50-54': 52.0, 
               '55-59': 57.0,'60-64': 62.0, '65-69': 67.0, '70-74': 72.0, '75-79': 77.0, '80+': 85.0}

# apply the mapping
data['Age_Category'] = data['Age_Category'].map(age_mapping)
data = data.rename(columns={'Age_Category':'Age_Numeric'})

# normalise the age (done in such a way that we can interpret an increase in age as a 10 year increase over the average age)
data['Age_Numeric'] = (data['Age_Numeric']- data['Age_Numeric'].mean())/10
data = data.rename(columns={'Age_Numeric':'Age_Normalised'})

# diabetes transformation
# group all kinds of diabetics together
diabetes_binary = [0 if i == 'No' else 1 for i in data['Diabetes']]
data['Diabetes'] = diabetes_binary
data = data.rename(columns={'Diabetes':'Diabetic'})

# group 'recent' checkups (within past 2 years)
checkup_binary = [1 if i == 'Within the past year' or i == 'Within the past 2 years' else 0 for i in data['Checkup']]
data['Checkup'] = checkup_binary
data = data.rename(columns={'Checkup':'Recent_Checkup'})

# transform perceived health
health_binary = [0 if i == 'Fair' or i == 'Poor' else 1 for i in data['General_Health']]
data['General_Health'] = health_binary
data = data.rename(columns={'General_Health':'Health_Good_or_Better'})

## Dropping consumption columns (without knowing exactly what their contents mean they are useless)

In [ ]:
data = data.drop(columns=['Alcohol_Consumption', 'Fruit_Consumption', 'Green_Vegetables_Consumption', 'FriedPotato_Consumption'])
data.head()

In [ ]:
# TODO
# calculate/plot correlation coefficients

## Plots for the bars/histograms based on if a person has heart disease or not, to possibly see correlations between a variable and heart disease (or not). It shows the difference between the group with and without heart disease.

In [ ]:
no_heart_disease = data.loc[data["Heart_Disease"] == 0]
heart_disease = data.loc[data["Heart_Disease"] == 1]

no_heart_disease_y = no_heart_disease["Heart_Disease"] # y vector
heart_disease_y = heart_disease["Heart_Disease"] # y vector

no_heart_disease = no_heart_disease.drop(columns=["Heart_Disease"])
heart_disease = heart_disease.drop(columns=["Heart_Disease"])

fig, axs = plt.subplots(5, 3, figsize=(25, 30))

c = 0
for i in range(14):

    plt.subplot(5, 3, i + 1)
    plt.hist(no_heart_disease[no_heart_disease.columns[i]], label='No heart disease', alpha=0.5, color='b')
    plt.hist(heart_disease[heart_disease.columns[i]], label='Heart disease', alpha=0.5, color='orange')
    plt.xlabel(no_heart_disease.columns[i])
    plt.ylabel('Frequency')
    plt.title(f'Data of {no_heart_disease.columns[i]} for people with and without heart disease')

    unique_vals = no_heart_disease[no_heart_disease.columns[i]].unique()
    if len(unique_vals) == 2 and all([x in [0, 1] for x in unique_vals]):
        plt.xticks([0, 1], ['No', 'Yes'])

    plt.legend()
    c += 1

# remove empty subplot
fig.delaxes(axs[-1, -1])
plt.show()

## Running a basic logistic regression model

In [ ]:
# splitting into X and Y and adding a constant
Y = data['Heart_Disease']
X = data.drop(columns='Heart_Disease')
X_intercept = sm.add_constant(X)

# recombining XY
column_names = ['constant']
for i in data.columns:
    column_names.append(i)

XY_constant = pd.DataFrame(np.hstack((np.array(Y)[:, np.newaxis], X_intercept)), columns=column_names)

In [ ]:
# splitting into test and train set (25:75)
num_samples = 0.25 * len(XY_constant)
row_ids = list(range(XY_constant.shape[0]))

# randomly select 25% of the row ids
selected_row_ids = random.sample(row_ids, round(num_samples))

# subset to create train and test
data_test = XY_constant.iloc[XY_constant.index.isin(selected_row_ids)]
data_train = XY_constant.iloc[~XY_constant.index.isin(selected_row_ids)]

In [ ]:
Y_train = data_train['Heart_Disease']
Y_test = data_test['Heart_Disease']
X_train = data_train.drop(columns='Heart_Disease')
X_test = data_test.drop(columns='Heart_Disease')

In [ ]:
model = sm.Logit(Y_train, X_train).fit()

In [ ]:
model.summary()

In [ ]:
# generate predictions
predictions = model.predict(X_test)
# transfrom predictions to binary
prediction_binary = (predictions > 0.5).astype(int)

In [ ]:
# compute accuracy score
accuracy = accuracy_score(y_true = Y_test, y_pred = prediction_binary)
print(accuracy)

### Simulating the coefficients

In [ ]:
# extract the coefficients and their covariance matrix from the logistic regression fit
beta_mean = model.params
beta_cov = model.cov_params()
# number of simulations
n_simulations = 10000

# simulate coefficients
simulated_betas = multivariate_normal.rvs(mean = beta_mean, cov = beta_cov, size = n_simulations)

# derive odds from log-odds coefficients 
simulated_betas_odds = np.exp(simulated_betas)

In [ ]:
# plot the distribution of each coefficient to assess and interpret them
for name, i in zip(X_test.columns, range(simulated_betas_odds.shape[1])):
    plt.figure()
    plt.hist(simulated_betas_odds[:, i], bins=30, color='skyblue')
    plt.axvline(x = np.median(simulated_betas_odds, axis = 0)[i], color = 'orange', label = f'Median {np.median(simulated_betas_odds, axis = 0)[i]:.2f}')
    plt.axvline(x = np.percentile(simulated_betas_odds, 5, axis = 0)[i], color = 'orange', label = f'5th Percentile {np.percentile(simulated_betas_odds, 5, axis = 0)[i]:.2f}', linestyle = '--')
    plt.axvline(x = np.percentile(simulated_betas_odds, 95, axis = 0)[i], color = 'orange', label = f'95th Percentile {np.percentile(simulated_betas_odds, 95, axis = 0)[i]:.2f}', linestyle = '--')
    plt.title(f'{name}')
    plt.xlabel('Coefficient Value')
    plt.ylabel('Frequency')
    plt.legend()
    plt.show()

### Interpretation of Coefficients

- The constant represents the baseline odds of having heart disease. This means that for a person with a poor or fair self perceived health status, who has not had a checkup in the past two years, who does not exercise, who does not have skin cancer or any other cancer, who does not have depression, who is not diabetic, who does not have arthritis, who is female, who is of average age, height, weight, and BMI, and who does not have a smoking history, the odds of having heart disease are 0.86 (0.83, 0.88). This indicates that the average person that falls into this category has a smaller chance of having heart disease than the chance of not having heart disease.

- For individuals who *do* exercise, the odds of having heart disease are 1.24 (1.21, 1.28) relative to those that do not. This means that

### Implementation of Principal Component Analysis (PCA)

## Find the amount of components needed to perform PCA with a specific percentage of variance kept.

In [ ]:
from sklearn.decomposition import PCA

X = data.drop(columns='Heart_Disease')

# Percentage of variance that we want to keep, should be between 0 and 0.99
percentage = 0.95
components = 0

# simulate a do-while loop
while True:

    pca = PCA(n_components=components)
    pca_data = pca.fit_transform(X)

    # if this is smaller than 0 we have kept at least percentage amount
    # of variance in the PCA.
    if percentage - sum(pca.explained_variance_ratio_) < 0:
        break

    components += 1

print(f'Number of components to use to keep {percentage * 100:.0f}% of the variance: {components}')

## The exact same procedure, but now in a plot:

In [ ]:
# Percentage of variance that we want to keep, should be between 0 and 0.99
percentage = 0.95
components = 0

# simulate a do-while loop
components = 0
ys = []
while True:

    pca = PCA(n_components=components)
    pca.fit_transform(X)

    # if this is smaller than 0 we have kept at least percentage amount
    # of variance in the PCA.
    ys.append(sum(pca.explained_variance_ratio_))
    if percentage - sum(pca.explained_variance_ratio_) < 0:
        break

    components += 1

print(f'Number of components to use to keep {percentage * 100:.0f}% of the variance: {components}')
plt.scatter(range(len(ys)), ys, marker='x', color='red')
plt.plot(range(len(ys)), ys, linestyle='--')
plt.xticks(range(len(ys)))
plt.axhline(percentage, color='green', linestyle='dashdot', label='Desired kept variance')
plt.xlabel('Number of components')
plt.ylabel('Kept variance (%)')
plt.legend()
plt.show()

### Implement Cross-Validation (k-fold)

In [ ]:
from sklearn.model_selection import KFold

# selecting the target variable
y = XY_constant['Heart_Disease']

# define K for cross validation
K = 100

# setup the k-fold cross-validation
kf = KFold(n_splits = K, shuffle = True)

# dictionary for the model names and features
# TODO: ensure PCA features are DataFrame, and ensure Y variable has same name
pca_df = pd.DataFrame(pca_data)
models = {'Base model': XY_constant.drop(columns='Heart_Disease'), 'PCA model': pca_df, 'Constant model': XY_constant['constant']}

# initialize a dictionary to store accuracy data and aic data
acc_dict = {key: [] for key in models.keys()}
aic_dict = {key: [] for key in models.keys()}

for key, df in models.items():

    for train_index, test_index in kf.split(XY_constant):
        
        # Split into train and test according to the folds 
        X_train, X_test = df.iloc[train_index], df.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # For each fold split, fit the model
        model = sm.Logit(y_train, X_train).fit()
                
        # Predict probabilities
        y_pred_prob = model.predict(X_test)

        # convert probabilities to binary predictions
        y_pred_binary = (y_pred_prob > 0.5).astype(int)

        acc_dict[key].append(accuracy_score(y_true = y_test, y_pred = y_pred_binary))
        aic_dict[key].append(model.aic)

In [ ]:
# calculate the mean accuracy of the baseline model
mean_acc = np.mean(acc_dict['Base model'])

#calculate average aic of all models
average_aic = {key: sum(aic)/len(aic) for key, aic in aic_dict.items()}

# plot the histogram of the PCA accuracies and overlay the baseline model's mean
plt.hist(acc_dict['PCA model'], bins=30, color='skyblue', label = 'PCA Accuracies')
plt.hist(acc_dict['Base model'], bins=30, color='teal', alpha=0.4, label='Complete model accuracies')
plt.hist(acc_dict['Constant model'], bins=30, color='firebrick', alpha=0.3, label='Constant model')
plt.title('Distribution of k-fold computed accuracies')
plt.xlabel('Accuracy')
plt.ylabel('Count')
plt.legend()
plt.show()

#print average aic scores
print('average aic:', average_aic)